In [1]:
import pandas as pd

### Fetch QC Symbols

- Note: When initializing database, make sure to get current values for these symbols
- Also, reconsider symbol filters applied when fetching QC symbols (its called filter_conditions_news)


In [3]:
qcSymbols = pd.read_csv('../StocksUniverse/QCnewsSymbols.csv', low_memory=False, on_bad_lines='warn')
print(f"# of QC Symbols: {len(qcSymbols)}")
qcSymbols.head(2)


# of QC Symbols: 2882


,Symbol,SymbolID,SymbolValue,Exchange,ShortName,PrimarySymbol,CompanyId,StandardName,LastPrice,LastVolume,...,SectorName,IndustryGroupName,IndustryName,SectorETF,IndustryGroupETF,IndustryETF,DollarVolume,QCSymbol,QCSymbolID,NewsSourceSymbol
0,FCNCA R735QTJ8XC9X,FCNCA R735QTJ8XC9X,FCNCA,NAS,First Citizens BancShares,FCNCA,0C000007MZ,First Citizens BancShares Inc,1437.48,152647,...,FinancialServices,Banks,BanksRegional,XLF,KBE,KRE,219000000.0,FCNCA,FCNCA R735QTJ8XC9X,FCNCA.BenzingaNews R735QTJ8XC9W
1,FMC R735QTJ8XC9X,FMC R735QTJ8XC9X,FMC,NYS,FMC,FMC,0C000007PR,FMC Corp,60.23,2496276,...,BasicMaterials,A,AgriculturalInputs,XLB,NaN,MOO,150000000.0,FMC,FMC R735QTJ8XC9X,FMC.BenzingaNews R735QTJ8XC9W


### Fetch News Data

In [4]:
df = pd.read_csv('../News/NewsQC.csv', low_memory=False, on_bad_lines='warn', thousands=',', index_col=0)
df.index = pd.to_numeric(df.index, errors='coerce').fillna(-1).astype(int)
df.head(2)

,symbol,time,author,categories,contents,createdat,symbols,tags,teaser,title,...,iE15m,iE1h,iE2m,iE30m,iE5m,interval_start_price_iETF,return_iETF,start_price_iETF,RelatedEmbeddings,RelatedScores
0,ESLTF R735QTJ8XC9X,2017-09-12 05:31:51-04:00,Charles Gross,"News,Contracts",U.S. Customs and Border Protection (CBP) award...,2017-09-12 05:31:51-04:00,ESLTF,NaN,U.S. Customs and Border Protection (CBP) award...,Elbit Systems of America Awarded Contract to P...,...,0.0,0.0,0.0,0,0,85.0,-0.235291,85.0,[],[]
1,HRS R735QTJ8XC9X,2017-09-12 05:35:39-04:00,Charles Gross,"News,Contracts",Harris Corporation (NYSE: HRS ) has been award...,2017-09-12 05:35:39-04:00,HRS,NaN,NaN,Harris Corporation Awarded Development Contrac...,...,0.0,0.0,0.0,0,0,85.0,-0.235291,85.0,['10043366_1'],[0.807]


### Find Missing Indices
 - Not able to copy these in batches like the rest (although can copy individually)

In [4]:
missing = sorted(set(range(df.index.min(), df.index.max() + 1)) - set(df.index[df.index != -1]))

if missing:
    print(f"Found {len(missing)} missing indices")
    print("First few missing indices:", missing[:10])
else:
    print("No gaps found - indices are sequential")

Found 527 missing indices
First few missing indices: [-1, 295462, 295475, 295476, 295477, 295478, 295479, 295480, 295481, 295482]


### ToDo:

1. Not fetching all symbols from Benzinga - probably restricting to QC Universe
2. While 146 industries, only 65 present in the data (maybe grouping them)
3. 

In [4]:
print(f"Length of df: {len(df)}")
print(f"Unique News Items: {df.unique_id.nunique()}")
print(f"Number of missing indices: {len(missing)}\n")

max_len = max(len(col) for col in df.columns)
print("# of unique items in:")
for col in df.columns:
    print(f"{col:{max_len}}: {df[col].nunique()}")

Length of df: 389779
Unique News Items: 389253
Number of missing indices: 527

# of unique items in:
symbol                   : 2845
time                     : 386691
author                   : 282
categories               : 8480
contents                 : 367892
createdat                : 386673
symbols                  : 2818
tags                     : 14180
teaser                   : 309442
title                    : 346582
market_session           : 5
Sector                   : 12
ETF                      : 12
Industry                 : 146
iETF                     : 65
unique_id                : 389253
updatedAfter             : 62
SymbolID                 : 2846
ShortName                : 2844
-2m                      : 153370
-5m                      : 180138
15m                      : 215069
1h                       : 263489
2m                       : 155722
30m                      : 238665
5m                       : 182630
V-2m                     : 177897
V-5m               

### Market Session
    'pre_market', 'in_market', 'post_market', 'market_closed'
    
1. 167286 index has 24676989_1 as market session
2. also has nan as market session

In [5]:
df.market_session.unique()

array(['pre_market', 'in_market', 'post_market', 'market_closed',
       '24676989_1', nan], dtype=object)

### Returns

In [6]:

pd.set_option('display.max_columns', None)  # Show all columns
# pd.set_option('display.max_rows', None)     # Show all rows
# pd.set_option('display.width', None)        # Width of the display in characters
# pd.set_option('display.max_colwidth', None) # Show full contents of cells

df.sort_values(by='price_return', ascending=False).head(100).tail(10)

,symbol,time,author,categories,contents,createdat,symbols,tags,teaser,title,market_session,Sector,ETF,Industry,iETF,unique_id,updatedAfter,SymbolID,ShortName,-2m,-5m,15m,1h,2m,30m,5m,V-2m,V-5m,V15m,V1h,V2m,V30m,V5m,end_price,end_time,end_volume,interval_start_price,interval_start_time,interval_start_volume,is_trading_day,originalTime,outlier,price_return,start_price,start_time,start_volume,volume_return,E-2m,E-5m,E15m,E1h,E2m,E30m,E5m,S-2m,S-5m,S15m,S1h,S2m,S30m,S5m,end_price_ETF,end_price_SPY,interval_start_price_ETF,interval_start_price_SPY,return_ETF,return_SPY,start_price_ETF,start_price_SPY,end_price_iETF,iE-2m,iE-5m,iE15m,iE1h,iE2m,iE30m,iE5m,interval_start_price_iETF,return_iETF,start_price_iETF,RelatedEmbeddings,RelatedScores
124511,CERE XJ24U3H4NQZP,2021-06-29 07:10:33-04:00,Vandana Singh,"Biotech,Long Ideas,News,Health Care,Small Cap,...",Cerevel Therapeutics Holdings Inc (NASDAQ: CER...,2021-06-29 07:10:33-04:00,CERE,"Briefs,Phase 1 Trial,Schizophrenia",NaN,Cerevel Therapeutics Stock Shoots Higher As Sc...,pre_market,Healthcare,XLV,Biotechnology,IBB,21761108_1,0.0,CERE XJ24U3H4NQZP,Cerevel Therapeutics Hldg,1.073911,-2.794648,3.999996,26.437496,1.250005,12.937498,3.125000,2.245336,0.890445,0.049736,1.928957,1.063861,1.458069,0.362217,22.850000,2021-06-29 09:35:00-04:00,913220,16,2021-06-29 07:11:00-04:00,16849.0,TRUE,2021-06-29 07:10:33-04:00,FALSE,42.810001,16.000000,2021-06-29 07:11:00-04:00,16849.0,54.200001,0.000000,-0.087303,0.000000,0.000000,0.000000,0.000000,0.000000,-0.014037,-0.037432,-0.002342,0.014043,-0.002342,-0.014043,-0.002342,126.099998,428.279999,125.889999,427.320007,0.166810,0.224654,125.889999,427.320007,163.339996,0.000000,0.000000,-0.134426,-0.177205,0.000000,-0.134426,0,163.660004,-0.195532,163.660004,['21760855_1' '21564470_1' '20593625_1' '18268...,[0.951 0.875 0.868 0.864 0.86 0.858 0.858 0.85...
83116,UPWK WYDQVEXITDNP,2020-11-04 16:32:37-05:00,Benzinga Newsdesk,"Earnings,News",Upwork (NASDAQ: UPWK ) reported quarterly earn...,2020-11-04 16:32:37-05:00,UPWK,NaN,Upwork (NASDAQ:UPWK) reported quarterly earnin...,Upwork Q3 Adj. EPS $0.04 Beats $(0.08) Estimat...,post_market,Industrials,XLI,StaffingAndEmploymentServices,XLY,18215880_1,0.0,UPWK WYDQVEXITDNP,Upwork,-0.653919,1.230137,4.556960,28.101261,1.265823,4.810131,2.430377,0.731638,0.220990,9.498070,69.521233,1.003861,1.540541,5.903475,28.160000,2020-11-05 09:35:00-05:00,231048,19.75,2020-11-04 16:33:00-05:00,259.0,TRUE,2020-11-04 16:32:37-05:00,FALSE,42.560001,19.750000,2020-11-04 16:33:00-05:00,259.0,892.080017,0.000000,-0.289679,0.101054,0.101054,0.000000,0.101054,0.000000,-0.026160,0.063980,0.217974,0.127876,0.049412,0.026166,0.122070,81.010002,349.329987,79.169998,344.079987,2.324116,1.525808,79.169998,344.079987,152.740005,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,150.289993,1.630190,150.289993,['16930713_1' '14749308_1' '15421572_1' '15971...,[0.991 0.99 0.989 0.987 0.932 0.922 0.921 0.92...
335739,CVNA WK2S2IM8SSPX,2023-07-19 06:33:42-04:00,Benzinga Newsdesk,"Earnings,News",Carvana (NYSE: CVNA ) reported quarterly sales...,2023-07-19 06:33:41-04:00,CVNA,NaN,Carvana (NYSE:CVNA) reported quarterly sales o...,Carvana Q2 Sales $2.97B Beat $2.59B Estimate,pre_market,ConsumerCyclical,XLY,AutoAndTruckDealerships,CARZ,33280995_1,1.0,CVNA WK2S2IM8SSPX,Carvana,-1.630860,2.453491,54.131577,28.500006,7.868425,27.657890,8.473687,0.680748,4.493696,3.009656,1.76934,2.135955,6.408046,2.909558,54.169998,2023-07-19 09:35:00-04:00,1391166.125,38,2023-07-19 06:34:00-04:00,27443.0,TRUE,2023-07-19 06:33:42-04:00,FALSE,42.549999,38.000000,2023-07-19 06:34:00-04:00,27443.0,50.689999,0.000000,0.000000,0.039697,0.079393,0.000000,0.039697,0.000000,-0.013202,0.000000,-0.004405,-0.024211,0.011003,0.022006,0.002193,176.740005,455.440002,176.339996,454.390015,0.226843,0.231076,176.339996,454.390015,59.520000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0,59.520000,0.000000,59.520000,['28363070_1' '31038109_1' '26725636_1' '32226...,[0.991 0.989 0.987 0.

### tags:
1. why it's moving, 

In [6]:
movingTags =["Why is it moving?", "why it's moving", "Why Is It Moving", ]

movingFeed = df[(df.tags.str.contains('moving', case=False, na=False)) | (df.categories == 'Movers') 
                | (df.teaser.str.contains('moving', case=False, na=False))
                | (df.title.str.contains('moving', case=False, na=False))
                ]

# movingFeed = df[df.select_dtypes(include=['object']).apply(lambda x: x.str.contains('moving', case=False, na=False)).any(axis=1)] # For any column (tags, contents, title etc)

print(f"# of movingFeed: {len(movingFeed)}")
print(f"# of Unique symbols: {len(movingFeed.symbol.unique())}")
movingFeed.sort_values(by='time', ascending=False).head(20).tail(5)


# of movingFeed: 9359
# of Unique symbols: 1538


,symbol,time,author,categories,contents,createdat,symbols,tags,teaser,title,...,iE15m,iE1h,iE2m,iE30m,iE5m,interval_start_price_iETF,return_iETF,start_price_iETF,RelatedEmbeddings,RelatedScores
389328,SGML XRT7W0J556CL,2023-12-18 15:43:15-05:00,Ryan Gustafson,"News,Movers",Sigma Lithium Corporation (NASDAQ: SGML ) shar...,2023-12-18 15:43:14-05:00,SGML,"BZ Data Project,why it's moving",Sigma Lithium Corporation (NASDAQ: SGML) share...,What's Going On With Sigma Lithium Stock?,...,-0.154704,-0.524282,-0.008595,-0.524282,0.025785,58.174999,-0.163299,58.174999,['36279940_1' '36115074_1' '31828450_1' '34287...,[0.922 0.905 0.905 0.904 0.903 0.903 0.902 0.8...
389272,TROX V7J1KKBMZMUD,2023-12-18 12:42:14-05:00,Lekha Gupta,"Analyst Color,Equities,Mid Cap,News,Upgrades,P...",Tronox Holdings PLC (NYSE: TROX ) shares are r...,2023-12-18 12:42:14-05:00,TROX,"Briefs,Expert Ideas,why it's moving",Tronox Holdings PLC (NYSE: TROX) shares are ri...,"Tronox Gains Favorable Upgrade From Analyst, P...",...,0.023556,-0.159067,0.035346,0,0.070691,84.870003,-0.188529,84.870003,['36278690_1' '36286901_1' '30908617_1' '33619...,[0.937 0.931 0.93 0.929 0.929 0.928 0.927 0.92...
389258,SNAP WIINL6RMFSKL,2023-12-18 12:13:14-05:00,Erica Kollmann,"Analyst Color,News,Social Media,Upgrades,Price...",Snap Inc. (NYSE: SNAP ) shares are trading hig...,2023-12-18 12:13:14-05:00,SNAP,"Expert Ideas,Guggenheim,Michael Morris,why it'...",Snap Inc. (NYSE: SNAP) shares are trading high...,Why Snap Shares Are Trending Higher Monday,...,0.000000,0.045812,0.027943,0.055861,0.027943,35.799999,0.027943,35.799999,['36095405_1' '29401093_1' '26762591_1' '27991...,[0.95 0.946 0.943 0.942 0.941 0.941 0.94 0.938...
389243,ILMN RWQR2INKP0TH,2023-12-18 11:55:04-05:00,Vandana Singh,"Large Cap,M&A,News,Health Care,General","On Sunday, gene-sequencing company Illumina In...",2023-12-18 11:55:04-05:00,ILMN,"Briefs,why it's moving","On Sunday, gene-sequencing company Illumina In...",Illumina-Grail $8B Saga Ends After It Battles ...,...,0.094414,0.566459,0.000000,0.302112,0.028324,52.959999,0.151062,52.959999,['35216323_1' '35144395_1' '28753986_1' '36202...,[0.97 0.969 0.957 0.955 0.951 0.948 0.945 0.92...
389242,NFLX SEWJWLJNHZDX,2023-12-18 11:52:30-05:00,Adam Eckert,"News,Price Target,Movers",Netflix Inc (NASDAQ: NFLX ) shares are trading...,2023-12-18 11:52:30-05:00,NFLX,"BZ Data Project,why it's moving",Netflix Inc (NASDAQ: NFLX) shares are trading ...,What's Going On With Netflix Stock?,...,0.069511,-0.208539,-0.076461,0.083411,-0.076461,71.930000,-0.291950,71.930000,['30526315_1' '31889125_1' '35327953_1' '28130...,[0.964 0.962 0.961 0.961 0.961 0.959 0.956 0.9...


In [45]:
8.76/6.8-1

0.28823529411764715

In [46]:
index = 10  
import textwrap
print(f"Title: {movingFeed.iloc[index].title}\n")
print(f"Returned: {movingFeed.iloc[index].price_return}%\n")
print(f"Teaser: {movingFeed.iloc[index].teaser}\n")
print(f"Contents: {textwrap.fill(movingFeed.iloc[index].contents, width=80)}\n")

Title: Marathon Patent Group Shares Up 17.4% Wed.; Co. Likely Moving In Sympathy With Blockchain/Crypto-Related Stocks; Co. Also Has Low Shares Outstanding Of 8.75M Shares

Returned: 28.82%

Teaser: Here are all of the company's patent portfolio: https://www.marathonpg.com/patent-portfolio/overview.As recently as Nov. 2, 2017, the company announced the purchase of Global Bit Ventures, a digital asset

Contents: Here are all of the company's patent portfolio:
https://www.marathonpg.com/patent-portfolio/overview. As recently as Nov. 2,
2017, the company announced the purchase of Global Bit Ventures, a digital asset
technology company that mines cryptocurrencies.
https://www.marathonpg.com/news/press-releases/detail/1166/marathon-patent-
group-to-acquire-global-bit-ventures-inc



### Categories

- Get **Macroeconomic Categories**

- **Company Specific Categories**
Guidance, Previews, Movers, Insider Trades, 'Econ #s', 'Signals', 'Events', 'Pre-Market Outlook', 'Initiation', 'Dividends', 'Reviews', 'Earnings Beats', 'Asset Sales', 'Earnings Misses', 'M&A', 'Price Target', 'Small Cap Analysis', 'Economics', 'Stock Split', 'Equities', 'Opinion', 'Earnings', 'Large Cap', 'Movers & Shakers', 'Exclusives', 'Downgrades', 'Market-Moving Exclusives', 'Press Releases', 'Trading Ideas', 'After-Hours Center', 'Short Ideas', 'Analyst Ratings', 'Intraday Update', 'Upgrades', 'Buybacks', 'Short Sellers', 'Long Ideas', 'Analyst Color'

In [27]:
# Unique categories
uniqueCategories = set([cat for cats in df.categories.unique().tolist() for cat in (cats.split(',') if isinstance(cats, str) else [])])
print(f"# of unique categories: {len(uniqueCategories)}")
print(uniqueCategories, sep='\n')

# of unique categories: 119
{'Wall Street Journal', 'Reiteration', 'Emerging Markets', 'Success Stories', 'Interview', 'Top Stories', 'Emerging Market ETFs', 'WIIM', 'Offerings', 'Management', 'Movers', 'Specialty ETFs', 'Guidance', 'Previews', 'Real Estate', 'Insider Trades', 'Econ #s', 'Fintech', 'Signals', 'Events', 'Media', 'Retail Sales', 'General', 'Gaming', 'Pre-Market Outlook', 'Small Business', 'Initiation', 'Dividends', 'Sports Betting', 'Fast Money', 'Insurance', 'Small Cap', 'Crowdsourcing', 'Asia', 'Reviews', 'Earnings Beats', 'Binary Options', 'Penny Stocks', 'Futures', 'Asset Sales', 'Earnings Misses', 'M&A', 'Technicals', 'Price Target', 'Federal Reserve', 'SEC', 'Financing', 'Restaurants', 'Eurozone', 'SPACE', 'Startups', 'Jim Cramer', 'Small Cap Analysis', 'Economics', 'New ETFs', 'Mid Cap', 'Stock Split', 'Equities', 'Opinion', 'Earnings', 'Large Cap', 'Social Media', 'ETFs', 'Politics', 'Movers & Shakers', 'Exclusives', 'Sector ETFs', 'Entrepreneurship', 'Downgrades